# CRP Functions

In [1]:
import numpy as np

import sys
from pathlib import Path

# Dynamically add DCM_utils directory to path
sys.path.insert(0, str(Path('..').resolve()))
from DCM_utils import *
from EulerRodriguesParameters import *

# 1) CRP_to_DCM

In [2]:
def CRP_to_DCM(q):
    """
    Converts a Classical Rodrigues Parameters (CRP) vector to a Direction Cosine Matrix (DCM).

    Args:
        q (np.array): A numpy array of size 3 representing the CRP vector (Gibbs vector).

    Returns:
        np.array: A 3x3 rotation matrix (DCM) corresponding to the rotation defined by the CRP vector.

    Notes:
        - The function assumes a passive rotation (coordinate transformation).
        - Ensure that q is a numpy array of floats for numerical precision.
    """
    # Validate input vector
    validate_vec3(q)
    
    # Ensure q is a numpy array of floats and reshape to make it a 3-element array
    q = np.array(q, dtype=np.float64).reshape(3)
    
    # Compute the skew-symmetric matrix q_cross (q^x)
    q_tilde = skew_symmetric(q)

    # Compute inner product: q^Tq (which is just squared magnitude of q)
    q_squared = np.dot(q, q)
    
    # Compute the outer product: qq^T
    q_outer = np.outer(q, q)
    
    # Identity matrix
    identity_matrix = np.eye(3)
    
    C = (1 / (1 + q_squared)) * ( ((1 - q_squared) * identity_matrix) + (2 * q_outer) - (2 * q_tilde) )
    
    return C

## 1.1 - Functional Testing of CRP_to_DCM

In [3]:
from pathlib import Path
import sys
import numpy as np

# Add the directory where RigidBodyKinematics.py is located to sys.path
path_to_rigid_body_kinematics = Path(r"..\..\Codes from AVS Lab")
sys.path.insert(0, str(path_to_rigid_body_kinematics))

# Import the gibbs2C function from the AVS lab
from RigidBodyKinematics import gibbs2C

# Define test CRP vectors (Classical Rodrigues Parameters)
test_crp_vectors = [
    [0, 0, 0],          # No rotation (identity rotation)
    [1, 0, 0],          # 180-degree rotation about x-axis
    [0, 1, 0],          # 180-degree rotation about y-axis
    [0, 0, 1],          # 180-degree rotation about z-axis
    [0.5, 0.5, 0],      # Rotation about the [1, 1, 0] axis
    [0.5, 0.5, 0.5],    # Rotation about the [1, 1, 1] axis
    [-0.5, 0.5, 0],     # Rotation about the [-1, 1, 0] axis
]

# Test each CRP vector
for i, crp in enumerate(test_crp_vectors):
    crp = np.array(crp, dtype=float)

    print(f"Test Case {i + 1}:")
    print(f"CRP vector = {crp}")

    # Compute DCM using the existing gibbs2C function
    C_existing = gibbs2C(crp)

    # Compute DCM using your CRP_to_DCM function
    C_custom = CRP_to_DCM(crp)

    # Ensure both are NumPy arrays for easy comparison
    C_existing = np.array(C_existing)
    C_custom = np.array(C_custom)

    # Calculate the difference between the two C matrices
    difference = C_existing - C_custom
    max_diff = np.max(np.abs(difference))

    # Print the results
    print(f"Max difference between CRP_to_DCM and gibbs2C: {max_diff:.12e}")
    if max_diff > 1e-12:
        print("C matrices differ significantly.\n")
    else:
        print("C matrices match.\n")

    print("-" * 50)


Test Case 1:
CRP vector = [0. 0. 0.]
Max difference between CRP_to_DCM and gibbs2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 2:
CRP vector = [1. 0. 0.]
Max difference between CRP_to_DCM and gibbs2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 3:
CRP vector = [0. 1. 0.]
Max difference between CRP_to_DCM and gibbs2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 4:
CRP vector = [0. 0. 1.]
Max difference between CRP_to_DCM and gibbs2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 5:
CRP vector = [0.5 0.5 0. ]
Max difference between CRP_to_DCM and gibbs2C: 5.551115123126e-17
C matrices match.

--------------------------------------------------
Test Case 6:
CRP vector = [0.5 0.5 0.5]
Max difference between CRP_to_DCM and gibbs2C: 1.110223024625e-16
C matrices match.

-------

# 2) DCM_to_CRP

In [4]:
def DCM_to_CRP(dcm):
    """
    Converts a Direction Cosine Matrix (DCM) to the Classical Rodrigues Parameters (CRP) vector.

    Args:
        dcm (np.array): A 3x3 rotation matrix representing the DCM.

    Returns:
        np.array: A 3-element array representing the CRP vector.

    Notes:
        - The function first converts the DCM to a quaternion.
        - Then computes the CRP vector by dividing the vector part of the quaternion by its scalar part.
        - Assumes that the quaternion uses the scalar-first convention.
        - The function handles passive rotations (coordinate transformations).

    Raises:
        ZeroDivisionError: If the scalar part of the quaternion is zero (singularity at 180 degrees).

    Example:
        >>> dcm = np.eye(3)
        >>> q = DCM_to_CRP(dcm)
        >>> print(q)
        [0. 0. 0.]
    """
    # Convert DCM to quaternion
    b = DCM_to_EP(dcm)  # b should be a 4-element array [q0, q1, q2, q3]

    # Ensure b is a numpy array
    b = np.array(b, dtype=np.float64)

    # Extract scalar and vector parts
    q0 = b[0]      # Scalar part
    q_vec = b[1:]  # Vector part [q1, q2, q3]

    # Check for division by zero to avoid singularity at 180 degrees
    if np.isclose(q0, 0.0):
        raise ZeroDivisionError("The scalar part of the quaternion is zero; cannot compute CRP.")

    # Compute CRP vector by dividing the vector part by the scalar part
    crp = q_vec / q0

    return crp

## 2.1 - Functional Testing of DCM_to_CRP

In [5]:
# Define test CRP vectors
test_crps = [
    [0.0, 0.0, 0.0],          # No rotation
    [1.0, 0.0, 0.0],          # Rotation about x-axis
    [0.0, 1.0, 0.0],          # Rotation about y-axis
    [0.0, 0.0, 1.0],          # Rotation about z-axis
    [0.5, 0.5, 0.0],          # Rotation about xy-plane
    [0.577, 0.577, 0.577],    # General rotation about [1,1,1]
]

# Test each CRP vector
for i, crp in enumerate(test_crps):
    crp = np.array(crp, dtype=float)
    print(f"Test Case {i + 1}:")
    print(f"Input CRP vector = {crp}")

    # Compute DCM from CRP using CRP_to_DCM function
    C = CRP_to_DCM(crp)

    # Recover CRP from DCM using DCM_to_CRP function
    crp_reconstructed = DCM_to_CRP(C)

    # Calculate the difference between the original and reconstructed CRP vectors
    difference = crp - crp_reconstructed
    max_diff = np.max(np.abs(difference))

    # Print the results
    print(f"Reconstructed CRP vector = {crp_reconstructed}")
    print(f"Max difference: {max_diff:.12e}")

    if max_diff > 1e-12:
        print("CRP vectors differ significantly.\n")
    else:
        print("CRP vectors match.\n")

    print("-" * 50)


Test Case 1:
Input CRP vector = [0. 0. 0.]
Reconstructed CRP vector = [0. 0. 0.]
Max difference: 0.000000000000e+00
CRP vectors match.

--------------------------------------------------
Test Case 2:
Input CRP vector = [1. 0. 0.]
Reconstructed CRP vector = [1. 0. 0.]
Max difference: 1.110223024625e-16
CRP vectors match.

--------------------------------------------------
Test Case 3:
Input CRP vector = [0. 1. 0.]
Reconstructed CRP vector = [0. 1. 0.]
Max difference: 1.110223024625e-16
CRP vectors match.

--------------------------------------------------
Test Case 4:
Input CRP vector = [0. 0. 1.]
Reconstructed CRP vector = [0. 0. 1.]
Max difference: 1.110223024625e-16
CRP vectors match.

--------------------------------------------------
Test Case 5:
Input CRP vector = [0.5 0.5 0. ]
Reconstructed CRP vector = [0.5 0.5 0. ]
Max difference: 0.000000000000e+00
CRP vectors match.

--------------------------------------------------
Test Case 6:
Input CRP vector = [0.577 0.577 0.577]
Reconst

# 3) Bmat_CRP

In [6]:
def Bmat_CRP(q):
    """
    Computes the 3x3 B matrix that relates the body angular velocity vector (ω) to the derivative 
    of the Classical Rodrigues Parameters (CRP) vector (q̇).

    The B matrix is defined as:

        B(q) = I + [q]× + q qᵗ

    where:
        - I is the 3x3 identity matrix.
        - [q]× is the skew-symmetric matrix of the vector q.
        - q qᵗ is the outer product of q with itself.

    The relationship between the CRP rates and the body angular velocity is given by:

        q̇ = (1/2) * B(q) * ω

    Args:
        q (array-like): A 3-element Classical Rodrigues Parameters (CRP) vector.

    Returns:
        np.ndarray: A 3x3 B matrix.

    Notes:
        - The function validates that `q` is a 3-element numeric vector.
        - It uses the skew-symmetric matrix computation for cross-product representation.

    Raises:
        ValueError: If the input vector `q` is not a valid 3-element numeric vector.
    """
    # Validate the input vector
    validate_vec3(q)

    # Convert input to a NumPy array
    q = np.array(q, dtype=float)

    # Compute the skew-symmetric matrix of q
    q_tilde = skew_symmetric(q)

    # Compute the B matrix
    B = np.eye(3) + q_tilde + np.outer(q, q)

    return B



Bmat_CRP([0,1,0])

array([[ 1.,  0.,  1.],
       [ 0.,  2.,  0.],
       [-1.,  0.,  1.]])

## 3.1 - Functional Testing of Bmat_CRP

In [7]:
from pathlib import Path
import sys
import numpy as np

# Add the directory where RigidBodyKinematics.py is located to sys.path
path_to_rigid_body_kinematics = Path(r"..\..\Codes from AVS Lab")
sys.path.insert(0, str(path_to_rigid_body_kinematics))

# Import the BmatGibbs function from the AVS lab
from RigidBodyKinematics import BmatGibbs

# Define test CRP vectors
test_crps = [
    [0.0, 0.0, 0.0],          # No rotation
    [1.0, 0.0, 0.0],          # Rotation about x-axis
    [0.0, 1.0, 0.0],          # Rotation about y-axis
    [0.0, 0.0, 1.0],          # Rotation about z-axis
    [0.5, 0.5, 0.0],          # Rotation about xy-plane
    [0.577, 0.577, 0.577],    # General rotation about [1,1,1]
]

# Test each CRP
for i, crp in enumerate(test_crps):
    crp = np.array(crp, dtype=float)
    print(f"Test Case {i + 1}:")
    print(f"CRP vector = {crp}")

    # Compute B matrix using the existing BmatGibbs function
    B_existing = BmatGibbs(crp)

    # Compute B matrix using your Bmat_CRP function
    B_custom = Bmat_CRP(crp)

    # Ensure both are NumPy arrays for easy comparison
    B_existing = np.array(B_existing)
    B_custom = np.array(B_custom)

    # Calculate the difference between the two B matrices
    difference = B_existing - B_custom
    max_diff = np.max(np.abs(difference))

    # Print the results
    print(f"Max difference between Bmat_CRP and BmatGibbs: {max_diff:.12e}")
    if max_diff > 1e-12:
        print("B matrices differ significantly.\n")
    else:
        print("B matrices match.\n")

    print("-" * 50)


Test Case 1:
CRP vector = [0. 0. 0.]
Max difference between Bmat_CRP and BmatGibbs: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 2:
CRP vector = [1. 0. 0.]
Max difference between Bmat_CRP and BmatGibbs: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 3:
CRP vector = [0. 1. 0.]
Max difference between Bmat_CRP and BmatGibbs: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 4:
CRP vector = [0. 0. 1.]
Max difference between Bmat_CRP and BmatGibbs: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 5:
CRP vector = [0.5 0.5 0. ]
Max difference between Bmat_CRP and BmatGibbs: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 6:
CRP vector = [0.577 0.577 0.577]
Max difference between Bmat_CRP and BmatGibbs: 0.000000000000e+00
B matrices match.

-

# 4) BInvmat_CRP

In [8]:
def BInvmat_CRP(q):
    """
    Computes the inverse B matrix (B_inv) that relates the derivative
    of the Classical Rodrigues Parameters (CRP) vector (q̇) to the body angular velocity vector (ω).

    The inverse B matrix is defined as:

        B_inv(q) = [I - [q]×] / (1 + qᵗ q)

    where:
        - I is the 3x3 identity matrix.
        - [q]× is the skew-symmetric matrix of the vector q.
        - qᵗ q is the dot product of q with itself (a scalar).

    The relationship between the body angular velocity and the CRP rates is given by:

        ω = 2 * B_inv(q) * q̇

    Args:
        q (array-like): A 3-element Classical Rodrigues Parameters (CRP) vector.

    Returns:
        np.ndarray: A 3x3 inverse B matrix.

    Notes:
        - The function validates that `q` is a 3-element numeric vector.
        - It uses the skew-symmetric matrix computation for cross-product representation.
        - The inverse B matrix is used to map the derivative of the CRP vector to the body angular velocity.

    Raises:
        ValueError: If the input vector `q` is not a valid 3-element numeric vector.
    """
    # Validate the input vector
    validate_vec3(q)

    # Convert input to a NumPy array
    q = np.array(q, dtype=float)

    # Compute the skew-symmetric matrix of q
    q_tilde = skew_symmetric(q)

    # Compute the denominator (1 + qᵗ q)
    denominator = 1 + np.dot(q, q)

    # Compute the inverse B matrix
    B_inv = (np.eye(3) - q_tilde) / denominator

    return B_inv

## 4.1 - Functional Testing of BInvmat_CRP

In [9]:
from pathlib import Path
import sys
import numpy as np

# Add the directory where RigidBodyKinematics.py is located to sys.path
path_to_rigid_body_kinematics = Path(r"..\..\Codes from AVS Lab")
sys.path.insert(0, str(path_to_rigid_body_kinematics))

# Import the BinvGibbs function from the AVS lab
from RigidBodyKinematics import BinvGibbs

# Define test CRP vectors
test_crps = [
    [0.0, 0.0, 0.0],          # No rotation
    [1.0, 0.0, 0.0],          # Rotation about x-axis
    [0.0, 1.0, 0.0],          # Rotation about y-axis
    [0.0, 0.0, 1.0],          # Rotation about z-axis
    [0.5, 0.5, 0.0],          # Rotation about xy-plane
    [0.577, 0.577, 0.577],    # General rotation about [1,1,1]
]

# Test each CRP
for i, crp in enumerate(test_crps):
    crp = np.array(crp, dtype=float)
    print(f"Test Case {i + 1}:")
    print(f"CRP vector = {crp}")

    # Compute B-inverse matrix using the existing BinvGibbs function
    B_inv_existing = BinvGibbs(crp)

    # Compute B-inverse matrix using your BInvmat_CRP function
    B_inv_custom = BInvmat_CRP(crp)

    # Ensure both are NumPy arrays for easy comparison
    B_inv_existing = np.array(B_inv_existing)
    B_inv_custom = np.array(B_inv_custom)

    # Calculate the difference between the two B-inverse matrices
    difference = B_inv_existing - B_inv_custom
    max_diff = np.max(np.abs(difference))

    # Print the results
    print(f"Max difference between BInvmat_CRP and BinvGibbs: {max_diff:.12e}")
    if max_diff > 1e-12:
        print("B-inverse matrices differ significantly.\n")
    else:
        print("B-inverse matrices match.\n")

    print("-" * 50)


Test Case 1:
CRP vector = [0. 0. 0.]
Max difference between BInvmat_CRP and BinvGibbs: 0.000000000000e+00
B-inverse matrices match.

--------------------------------------------------
Test Case 2:
CRP vector = [1. 0. 0.]
Max difference between BInvmat_CRP and BinvGibbs: 0.000000000000e+00
B-inverse matrices match.

--------------------------------------------------
Test Case 3:
CRP vector = [0. 1. 0.]
Max difference between BInvmat_CRP and BinvGibbs: 0.000000000000e+00
B-inverse matrices match.

--------------------------------------------------
Test Case 4:
CRP vector = [0. 0. 1.]
Max difference between BInvmat_CRP and BinvGibbs: 0.000000000000e+00
B-inverse matrices match.

--------------------------------------------------
Test Case 5:
CRP vector = [0.5 0.5 0. ]
Max difference between BInvmat_CRP and BinvGibbs: 0.000000000000e+00
B-inverse matrices match.

--------------------------------------------------
Test Case 6:
CRP vector = [0.577 0.577 0.577]
Max difference between BInvmat_